## Given no local transmission (or other arbitrary low level of transmission) in a single city, what is the effect of three days of 10^4+ people gathering for a hypothetical protest rally?

The following code seeks to answer this question by first running a bunch of simualtions tuneds to historical data for Australian cities (in this case, Sydney, Melbourne, Brisbane and Perth). This gives an estimate of the number of undetected exposed individuals which we use to run scenarios of mass gatherings for a few days (protests) vs. not. 

First, define some necessary epidemeological parameters, demographic parameters, and Julia packages. 

In [1]:
cd("/Users/michael/work/GitHub/epinets")
include("EpiSim.jl")
using CSV
using Plots
using LightGraphs
using JLD2, FileIO
#demographics
pops=[426709 8089526 245869 5095100 1751693 534281 6594804 2621680]
cpop=[447457 4741874 132708 2326656 1315346 208324 4677157 2004696]
cinf=[107 3110 29 1061 440 228 1681 596]
#'reasonable' parameters
epiparam=Dict()
epiparam["p0"]=0.2 #a guess - tuned to match observed data 
epiparam["p2"]=1/12 #revised infection rate with distancing measure
epiparam["q"]=1/7 #"up to" two weeks
epiparam["r0"]=1/14 #about two weeks for mild, 3-6 for severe
epiparam["r2"]=1/4 #revised removal rate (now due to testing and isolation)
epiparam["nseeds"]=5 #probably too many, consider dropping.
allstates=["Australian Capital Territory" "New South Wales"  "Northern Territory" "Queensland" "South Australia" "Tasmania" "Victoria" "Western Australia"]
allcities=["Canberra" "Sydney" "Darwin" "Brisbane" "Adelaide" "Hobart" "Melbourne" "Perth"]



1×8 Array{String,2}:
 "Canberra"  "Sydney"  "Darwin"  …  "Hobart"  "Melbourne"  "Perth"

Next, we run the simulations many, many times to get the distribution of background cases. This has already been done, the following cell can be skipped if you have the data stored in saved files (JLD files with the name of each city).

In [ ]:
cd("/Users/michael/work/GitHub/COVID-19/csse_covid_19_data/csse_covid_19_time_series")
file="time_series_covid19_confirmed_global.csv"
z=[]
for state in allstates
    z=push!(z,EpiSim.getdata("Australia",state))
end
ddays=CSV.File(file)[1]
ddays=propertynames(ddays,4)[5:end]
ddays=String.(ddays)
ndays=length(ddays)

for i in [2, 4, 7, 8]
	epiparam["pop"]=pops[i]
	epiparam["gridsize"]=Int(floor(sqrt(pops[i])))
	y=z[i];
	ddt(z,zt)=count(z->z>0, z[1:zt])+ count(z->z<0, z[zt+1:end])
	~,tpday=findmax([ddt(diff(diff(y)),nx) for nx in 1:(ndays-2)])
	#this is the turning point between exponential growth and decay. totItp total infections at day tpday
	totItp=y[tpday+1]
	plot(1:tpday+1,y[1:tpday+1],lw=4,label="growth phase",title=allstates[i])
	plot!(tpday+1:ndays,y[tpday+1:ndays],lw=4,label="plateau")
	cd("/Users/michael/work/GitHub/epinets")
	gridsize=epiparam["gridsize"]
	pop=epiparam["pop"]
	bamodel=barabasi_albert(gridsize^2, 3, 2)
	#lattice=LightGraphs.grid((gridsize,gridsize),periodic=true)
	#wattstrog95=watts_strogatz(gridsize^2, 4, 0.013)  #s=0.013 => 95% compliance
	#wattstrog90=watts_strogatz(gridsize^2, 4, 0.026)  #s=0.026 => 90% compliance
	wattstrog80=watts_strogatz(gridsize^2, 4, 0.053)  #s=0.053 => 80% compliance
	#St95,Et95,It95,Rt95=EpiSim.episim(bamodel,wattstrog95, epiparam, totItp, tpday+2, 90, 100)
	#St90,Et90,It90,Rt90=EpiSim.episim(bamodel,wattstrog90, epiparam, totItp, tpday+2, 90, 100)
	
	th=cinf[i]

	St80,Et80,It80,Rt80=EpiSim.episim(bamodel,wattstrog80, epiparam, totItp, tpday+2, 90, 100)
	(Sp,Ep,Ip,Rp) = EpiSim.epipred(th, St80,Et80,It80,Rt80)


	iso=0.8 #was 0.95, I think 0.8 should be enough ... actually this parameter *may* significantly affect the results as it determines the susceptible pool size
	pop=epiparam["pop"]
	grd=Int(floor(sqrt(Int(floor(pop*(1-iso))))))
	isosize=pop-grd^2
	sg=SimpleGraph(isosize)
	ws=watts_strogatz(grd^2, 4, 0.026)
	isograph80=union(sg,ws)
	St9,Et9,It9,Rt9=EpiSim.episim3(bamodel,wattstrog80, isograph80, epiparam, totItp, tpday+2, 1, 180, 400)
	(Sp,Ep,Ip,Rp) = EpiSim.epipred(th, St9,Et9,It9,Rt9)

	RR = Rp[2:end,:] - Rp[1:end-1,:] ### daily increase of removed
	II = RR+Ip[2:end,:]-Ip[1:(end-1),:] ### daily increase of infected
	EE = II+Ep[2:end,:]-Ep[1:(end-1),:] ### daily increase of exposed
	ET = Ep[2:end,:] ### current number of exposed

    flnm=["protest-"*allcities[i]]
	@save flnm RR II EE ET

end

100.0%┣█████████████████████████████████████┫ 100/100 [01:05:38<00:00, 0.0 it/s]
34.8%┣████████████▏                      ┫ 139/400 [01:30:33<02:51:16, 0.0 it/s]

In [ ]:
	cd("/Users/michael/work/GitHub/epinets")

In [ ]:
plts=Array{Any,1}(undef,8)
cld=Array{Any,1}(undef,8)
for i in [2 4 7 8]
    cld[i]=load(allcities[i])
    plts[i]=plot(cld[i]["II"],cld[i]["ET"],seriestype=:scatter,legend=false,title=allcities[i],xlabel="new infections", ylabel="total exposed",xlim=(0,100),ylim=(0, 1200))
end

In [ ]:
plot(plts[2], plts[4], plts[7], plts[8])

In [ ]:
function makeheatmappdf(II,ET)
    ninf=40
    texp=1000
    hidden=zeros(Int64, texp+1, ninf+1)
    (ndays,nsims)=size(II)
    for i in 1:ndays
        for j in 1:nsims
            xi=II[i,j]
            xj=ET[i,j]
            if xi<ninf && xj<texp
                hidden[xj+1,xi+1] += 1
            end
        end
    end
    hidpdf=Array{Float64, 2}(undef, texp+1, ninf+1)
    for i in 1:(ninf+1)
        hidpdf[:, i] = hidden[:, i] / sum(hidden[:, i])
    end
    return hidpdf
end


Finally, we arrive at the distribution of expected number of undiscloses exposed individuals on a day with a given number of new infections. I suspect the model fit for Sydney and Brisbane may not be great as those numbers look a little high.

In [ ]:
hidpdf=Array{Any,1}(undef,8)
ylims=[NaN 600 NaN 600 NaN NaN 300 300]
for i in [2 4 7 8]
    hidpdf[i]=makeheatmappdf(cld[i]["II"],cld[i]["ET"])
    plts[i]=heatmap(0:40,0:1000, hidpdf[i], title=allcities[i], xlabel="number of new infected", ylabel="number of hidden exposed",ylim=(0,ylims[i]), xlim=(0,20), color=cgrad([:white,:orange, :red], [0, 0.5, 0.8]) )
end
plot(plts[2], plts[4], plts[7], plts[8])

In [ ]:
i=8
plot(hidpdf[i][:,1:5],xlim=(0,100),linetype=:bar,xlabel="Number of undiagnosed exposed individuals (given N infections)",ylabel="pdf",title="Perth",label=["N=0" "N=1" "N=2" "N=3" "N=4"])

In [ ]:
using Random, StatsBase, Distributions, Statistics

In [ ]:
i=8
nedg,~=size(hidpdf[i])
randi=sample(0:(nedg-1), Weights(hidpdf[i][:,1]),10);

In [ ]:
ppop=[NaN 20 NaN 30 NaN NaN 10 5].*1000


In [ ]:
i=8
nsim=30
#'reasonable' parameters
epiparam=Dict()
epiparam["p0"]=1/10 #a guess - tuned to match observed data 
epiparam["q"]=1/7 #"up to" two weeks
epiparam["r0"]=1/4 #about two weeks for mild, 3-6 for severe
randi=sample(0:(nedg-1), Weights(hidpdf[i][:,1]),nsim);
epiparam["nseeds"]=randi; #probably too many, consider dropping.
epiparam["pop"]=cpop[i]



In [ ]:
include("EpiSim.jl")
include("ContactModels.jl")

In [ ]:
net1=covidsafe(limitmix(cpop[i],ppop[i]),0.5) #covidsafe at 50% and mass gathering of size popp[i]
net2=covidsafe(limitmix(cpop[i],50),0.5) #covidsafe at 50% and more modest 50 ppl gatherings
net3=covidsafe(nomassmix(cpop[i]),0.5) #covidsafe at 50% and no mass gatherings gatherings

In [ ]:
 st1, ex1, fe1, rm1 = EpiSim.episimprotest(net1,net3,epiparam,3,147,nsim) 
 st2, ex2, fe2, rm2 = EpiSim.episimprotest(net3,net3,epiparam,3,147,nsim) 

In [ ]:
plot(title=[allcities[i] " " ppop[i] " person protests over one weekend"])
EpiSim.plotquantiles(fe1+rm1,:red,"Protests",0.25)
EpiSim.plotquantiles(fe2+rm2,:blue,"No Protests",0.25)

In [ ]:
plotly()

In [ ]:
nppl=Array{Int,1}(undef, 8)
nppl[2]=20
nppl[4]=20
nppl[7]=20
nppl[8]=100
for i in [2] #[2 4 7 8]
    nsim=100
    randi=sample(0:(nedg-1), Weights(hidpdf[i][:,1]),nsim);
    epiparam["nseeds"]=randi; 
    epiparam["pop"]=cpop[i]
    net1=covidsafe(limitmix(cpop[i],ppop[i]),0.5) #covidsafe at 50% and mass gathering of size popp[i]
    net2=covidsafe(limitmix(cpop[i],nppl[i]),0.5) #covidsafe at 50% and more modest 50 ppl gatherings
    st1, ex1, fe1, rm1 = EpiSim.episimprotest(net1,net2,epiparam,3,147,nsim) 
    st2, ex2, fe2, rm2 = EpiSim.episimprotest(net2,net2,epiparam,3,147,nsim)
    
    filename="pro_"*allcities[i]
    @save filename st1 ex1 fe1 rm1 st2 ex2 fe2 rm2 net1 net2 epiparam randi nsim
end


In [ ]:
plotly()

In [ ]:
i=2
    filename="pro_"*allcities[i]
    ld=load(filename)

    plts[i]=plot(title=[allcities[i] " " ppop[i] " person protests over one weekend"])
    EpiSim.plotquantiles(ld["fe1"]+ld["rm1"],:red,"Protests",0.25)
    EpiSim.plotquantiles(ld["fe2"]+ld["rm2"],:blue,"No Protests",0.25)
for i in [4 7 8]
    filename="pro_"*allcities[i]
    ld=load(filename)

    plts[i]=plot(title=[allcities[i] " " ppop[i] " person protests over one weekend"])
    EpiSim.plotquantiles(ld["fe1"]+ld["rm1"],:red,0.25)
    EpiSim.plotquantiles(ld["fe2"]+ld["rm2"],:blue,0.25)
end

In [ ]:
plot(plts[2], plts[4], plts[7], plts[8],size=(800,700))

In [ ]:
i=2
    filename="pro_"*allcities[i]
    ld=load(filename)
    nd=60
    plot()
    plts[i]=histogram([ld["fe1"][nd,:]+ld["rm1"][nd,:], ld["fe2"][nd,:]+ld["rm2"][nd,:]], xlabel="New infections $(nd) days later",labels=false,bins=20, opacity=0.5,title=allcities[i])
for i in [4 7 8]
    filename="pro_"*allcities[i]
    ld=load(filename)
    nd=60
    plot()
    plts[i]=histogram([ld["fe1"][nd,:]+ld["rm1"][nd,:], ld["fe2"][nd,:]+ld["rm2"][nd,:]], xlabel="New infections $(nd) days later",labels=false,bins=20, opacity=0.5,title=allcities[i])
end

In [ ]:
plot(plts[2], plts[4], plts[7], plts[8],size=(800, 800))

In [ ]:
histogram([ld["fe1"][nd,:]+ld["rm1"][nd,:], ld["fe2"][nd,:]+ld["rm2"][nd,:]], xlabel="New infections $(nd) days later",labels=false,bins=20, opacity=0.5,title=allcities[i])

In [ ]:
st=Array{Any,1}(undef,42)
ex=Array{Any,1}(undef,42)
fe=Array{Any,1}(undef,42)
rm=Array{Any,1}(undef,42)
i=8
    nsim=100
 #   st0, ex0, fe0, rm0 = EpiSim.episimprotest(net2,net2,epiparam,3,147,nsim)
for j=1:42
    randi=sample(0:(nedg-1), Weights(hidpdf[i][:,1]),nsim);
    epiparam["nseeds"]=randi; 
    epiparam["pop"]=cpop[i]
    net1=covidsafe(limitmix(cpop[i],ppop[i]),0.5) #covidsafe at 50% and mass gathering of size popp[i]
    net2=covidsafe(limitmix(cpop[i],nppl[i]),0.5) #covidsafe at 50% and more modest 50 ppl gatherings
    st1, ex1, fe1, rm1 = EpiSim.episimprotest(net1,net2,epiparam,j,150-j,nsim) 
    st[j]=deepcopy(st1)
    ex[j]=deepcopy(ex1)
    fe[j]=deepcopy(fe1)
    rm[j]=deepcopy(rm1)
    @save "progress-ndays" st ex fe rm
end


In [ ]:
np=20
nd=30
i=8
histogram([fe[np][nd,:]+rm[np][nd,:], fe0[nd,:]+rm0[nd,:]], xlabel="New infections $(nd) days after $(np) days of proterst",labels=false,bins=20, opacity=0.5,title=allcities[i])



In [ ]:
@save("protest-ndays")

In [ ]:
fe[1]